In [2]:
import torch
import torch.nn as nn
import torch.functional as F
import torchvision.models as models

In [3]:
class Hybrid_model(nn.Module):

    def __init__(self,num_of_classes = 10):

        super(Hybrid_model, self).__init__()

        self.vgg16 = models.vgg16(weights = 'VGG16_Weights.DEFAULT')
        self.vgg19 = models.vgg19(weights = 'VGG19_Weights.DEFAULT')
        self.resnet18 = models.resnet18(weights = 'ResNet18_Weights.DEFAULT')
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))

        self.fc = nn.Linear(512*3, num_of_classes)

    def forward(self, x):

        layers = []

        x1 = self.vgg16.features(x)
        x1 = self.avgpool(x1)
        x1 = torch.flatten(x1, 1)

        layers.append(x1)

        x2 = self.vgg16.features(x)
        x2 = self.avgpool(x2)
        x2 = torch.flatten(x2, 1)

        layers.append(x2)

        x3 = self.resnet18.conv1(x)
        x3 = self.resnet18.bn1(x3)
        x3 = self.resnet18.relu(x3)
        x3 = self.resnet18.maxpool(x3)

        x3 = self.resnet18.layer1(x3)
        x3 = self.resnet18.layer2(x3)
        x3 = self.resnet18.layer3(x3)
        x3 = self.resnet18.layer4(x3)
        x3 = self.avgpool(x3)
        x3 = torch.flatten(x3, 1)

        layers.append(x3)

        x = torch.cat((x1, x2, x3), dim=1)
        x = self.fc(x)
        return x, layers



In [4]:
hybrid_model = Hybrid_model(num_of_classes=10)
print(hybrid_model)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:08<00:00, 68.4MB/s]
Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth
100%|██████████| 548M/548M [00:07<00:00, 77.8MB/s]
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 160MB/s]


Hybrid_model(
  (vgg16): VGG(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inplace=True)
      (

In [5]:
input_tensor = torch.rand(1, 3, 224, 224)
# Get intermediate layer outputs
x, layer_outputs = hybrid_model(input_tensor)

# Print layer outputs
for i, output in enumerate(layer_outputs):
    print(f"Output after layer {i + 1}: {output.shape}")

print("Final layer output: ", x.shape)

Output after layer 1: torch.Size([1, 512])
Output after layer 2: torch.Size([1, 512])
Output after layer 3: torch.Size([1, 512])
Final layer output:  torch.Size([1, 10])


In [6]:
total_params = sum(p.numel() for p in hybrid_model.parameters())
print(f"Total Parameters: {total_params}")

Total Parameters: 293729666


In [7]:
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam

In [8]:
tranform_train = transforms.Compose ([
                                      transforms.Resize([224,224]),
                                      transforms.RandomRotation(35),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2434, 0.2616))
                                    ])

tranform_test = transforms.Compose ([
                                      transforms.Resize([224,224]),
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2434, 0.2616))
                                    ])

train_data = datasets.CIFAR10(root = '.\CIFAR10', train = True,transform = tranform_train,download=True)

test_data  = datasets.CIFAR10(root = '.\CIFAR10', train = False,transform = tranform_test,download=True)

classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

100%|██████████| 170498071/170498071 [00:03<00:00, 47599211.04it/s]


Extracting .\CIFAR10/cifar-10-python.tar.gz to .\CIFAR10
Files already downloaded and verified


In [9]:
## Dataloader
## Loads batch of images = 32 in every iteration

batch_size = 32
traindataloader = DataLoader(train_data, batch_size=batch_size, shuffle = True)
testdataloader  = DataLoader(test_data,  batch_size=batch_size, shuffle = True)

In [10]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"using the device :{device}")

using the device :cuda


In [11]:
loss_fn = nn.CrossEntropyLoss()
opt = Adam(hybrid_model.parameters(),lr  = 0.001)

In [12]:
def train_batch(epoch, model, loss_fn, optimizer):

    print("epoch ", epoch)
    model.to(device)
    model.train()
    train_loss = 0
    correct = 0
    total = 0

    for batch_idx, (X, y) in enumerate(traindataloader):

        X, y = X.to(device), y.to(device)

        #initialise the gradients to zero
        optimizer.zero_grad()

        #predictions
        y_pred = model(X)

        #compute the loss
        loss = loss_fn(y_pred[0], y)

        #backpropogation
        loss.backward()

        #update the weights
        optimizer.step()

        #summ the loss

        train_loss += loss.item()
        _, predicted = y_pred[0].max(1)
        total += y.size(0)
        correct += predicted.eq(y).sum().item()
    print(batch_idx, len(traindataloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'% (train_loss/(batch_idx+1), 100.*correct/total, correct, total))



In [13]:
def test_batch(epoch, model, loss_fn):

    model.to(device)
    model.eval()
    test_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():

        for batch_idx, (X,y) in enumerate(testdataloader):

            X, y = X.to(device), y.to(device)

            y_pred = model(X)

            loss = loss_fn(y_pred[0],y)

            test_loss+=loss

            predicted = torch.argmax(y_pred[0],axis=1)

            total += y.size(0)

            correct += (predicted == y).type(torch.float).sum().item()

        print("No of batches", len(testdataloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'% (test_loss/(batch_idx+1), (correct/total)*100, correct, total))

In [14]:
epochs = 2
for epoch in range(epochs):

    train_batch(epoch, hybrid_model, loss_fn, opt)

    test_batch(epoch, hybrid_model, loss_fn)

print("Done!")

epoch  0
1562 1563 Loss: 0.842 | Acc: 71.094% (35547/50000)
No of batches 313 Loss: 0.546 | Acc: 81.390% (8139/10000)
epoch  1
1562 1563 Loss: 0.551 | Acc: 81.086% (40543/50000)
No of batches 313 Loss: 0.441 | Acc: 85.090% (8509/10000)
Done!


In [15]:
assets_dir = './sample_data'
model_path = assets_dir + 'hybrid_model.pt'
# Save the model's state dictionary to the specified path
torch.save(hybrid_model.state_dict(), model_path)

In [16]:
# Load the model
file_path = assets_dir + 'hybrid_model.pt'

# NOTE: Need to run the cells for HybridModel in case you are starting off with this cell

# Load the saved state dictionary
saved_state_dict = torch.load(file_path)

# Load the state dictionary into the model
hybrid_model.load_state_dict(saved_state_dict)

# Set the model to evaluation mode
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
hybrid_model = hybrid_model.to(device)
hybrid_model.eval()

Hybrid_model(
  (vgg16): VGG(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inplace=True)
      (

In [17]:
epoch = 1
test_batch(epoch, hybrid_model,loss_fn)

No of batches 313 Loss: 0.441 | Acc: 85.090% (8509/10000)
